In [2]:
import os
import pandas as pd
import numpy as np
import math
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import statsmodels.formula.api as smf
import statsmodels.api as sm
pd.set_option('display.max_colwidth', None)

In [5]:
multip_path = "/home/local/VANDERBILT/litz/data/nlst/DeepLungScreening/pred/ipn_pred_v2.csv"
cohort_path = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/nlst_ipn_v2.csv"
# multip_path = "/home/local/VANDERBILT/litz/data/vlsp/DeepLungScreening/pred/vlsp_pred_imp_v1.csv"
# cohort_path = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/vlsp_cs_imp_v1.csv"
multip = pd.read_csv(multip_path)
cohort = pd.read_csv(cohort_path, dtype={'pid':str})
# multip = multip[~multip['pred'].isnull()] # prediction may have failed resulting in NaN

In [8]:
multip[multip['pred'].isnull()]
multip = multip[~multip['pred'].isnull()]

In [9]:
df = multip.merge(cohort, on='id')
df = df.loc[df.groupby('pid')['session'].idxmax()]
df

,id,pred,pid,with_image,with_marker,age,gender,race,education,bmi,...,norm_pky,spic,upper,nodule_type,nodule_count,lung_cancer,norm_bmi,norm_logsize,subjwithkaggle,subjwithfactor
0,100004time2000,0.011661,100004,True,True,60,1,1,4,29.414135,...,0.017391,0,1,2.0,1,0,0.274569,0.076330,1,1
1,100012time2000,0.198917,100012,True,True,61,2,1,6,22.240116,...,0.030435,1,1,0.0,1,1,0.128352,0.427029,1,1
2,100019time2000,0.040926,100019,True,True,61,1,1,4,23.962608,...,0.208696,1,1,2.0,1,0,0.163459,0.408723,1,1
3,100026time2001,0.022328,100026,True,True,57,1,1,3,35.146505,...,0.136957,0,0,2.0,2,0,0.391403,0.135536,1,1
4,100035time2001,0.011342,100035,True,True,55,2,1,3,22.096473,...,0.034783,0,0,2.0,1,0,0.125424,0.135536,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,218391time2000,0.247947,218391,True,True,66,1,1,6,23.674792,...,0.043478,0,1,1.0,4,1,0.157593,0.460238,1,1
5756,218499time1999,0.071401,218499,True,True,63,1,1,2,23.110395,...,0.176087,1,1,1.0,1,1,0.146090,0.367822,1,1
5757,218510time2001,0.253550,218510,True,True,64,1,1,6,27.802713,...,0.176087,0,1,2.0,4,1,0.241726,0.408723,1,1
5758,218705time2001,0.030812,218705,True,True,68,1,1,4,37.305733,...,0.234783,0,1,2.0,2,0,0.435411,0.183911,1,1


In [10]:

# accuracy_score
y = df['lung_cancer']
y_prob = df['pred']
y_hat = round(df['pred'])
roc = roc_auc_score(df['lung_cancer'], df['pred'])
report = classification_report(y, y_hat)
acc = accuracy_score(y, y_hat)
print(f"AUC: {roc}")
print(report)
print(f"Accuracy: {acc}")

AUC: 0.8816048992611106
              precision    recall  f1-score   support

           0       0.88      1.00      0.94      5089
           1       0.00      0.00      0.00       671

    accuracy                           0.88      5760
   macro avg       0.44      0.50      0.47      5760
weighted avg       0.78      0.88      0.83      5760

Accuracy: 0.8835069444444444


/home-nfs2/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home-nfs2/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home-nfs2/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. U

In [11]:
features = [
    'age', 'race', 'education',  'bmi',  'copd', 'phist', 'fhist', 'smo_status', 'quit_time', 'pkyr', 
]
cohort['lung_cancer'] = cohort['lung_cancer'].astype(int)
features_df = cohort[features]
x = pd.get_dummies(features_df, columns=['race', 'education'])
enc_features = x.columns
# Multiple linear imputation
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(x)
imp_x = pd.DataFrame(imp.transform(x), columns=enc_features)
imp_x[['bmi', 'copd', 'phist', 'fhist']] = imp_x[['bmi', 'copd', 'phist', 'fhist']].round()

# merge imputed values with cohort labels using an index join
cohort_enc = pd.merge(cohort.drop(columns=['bmi', 'copd', 'phist', 'fhist']), imp_x[['bmi', 'copd', 'phist', 'fhist']], left_index=True, right_index=True)

In [12]:
# min-max normalization
# scalars = ['age', 'bmi', 'quit_time', 'pkyr', 'vol_all', 'vol_LLL', 'vol_LUL', 'vol_RLL', 'vol_RML', 'vol_RUL', 'lavp_all', 'lavp_LLL', 'lavp_LUL', 'lavp_RUL', 'lavp_RML', 'lavp_RLL']
# cohort_enc = cohort # if not imputing
scalars = ['age', 'bmi', 'quit_time', 'pkyr']
cohort_enc[scalars] = cohort_enc[scalars].astype(float)
cohort_enc[scalars].min()
cohort_enc[scalars] = (cohort_enc[scalars] - cohort_enc[scalars].min())/(cohort_enc[scalars].max() - cohort_enc[scalars].min())

In [13]:
cohort_enc['lung_cancer'].value_counts()

0    5113
1     676
Name: lung_cancer, dtype: int64

In [14]:
import statsmodels.formula.api as smf
# formula_str = 'age+bmi+copd+phist+fhist+smo_status+quit_time+pkyr+C(race, Treatment(reference=1))+C(education, Treatment(reference=1))'
formula_str = 'age+bmi+copd+phist+fhist+smo_status+quit_time+pkyr'
smf_lr = smf.logit(f"lung_cancer ~ {formula_str}", data=cohort_enc).fit()
smf_lr.summary()

Optimization terminated successfully.
         Current function value: 0.343025
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            lung_cancer   No. Observations:                 5789
Model:                          Logit   Df Residuals:                     5780
Method:                           MLE   Df Model:                            8
Date:                Tue, 08 Nov 2022   Pseudo R-squ.:                 0.04833
Time:                        19:16:51   Log-Likelihood:                -1985.8
converged:                       True   LL-Null:                       -2086.6
Covariance Type:            nonrobust   LLR p-value:                 2.802e-39
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.2984      0.174    -13.218      0.000      -2.639      -1.958
age            1.2801      0.154      8.290      0.000       0.977       1.583
bmi           -1.9774      0.469     -4.213      0.000      -2.897      -1.057
copd           0.5032      0.149      3.370      0.001       0.211       0.796
phist          0.4710      0.165      2.860      0.004       0.148       0.794
fhist          0.1943      0.098      1.981      0.048       0.002       0.386
smo_status     0.0234      0.121      0.193      0.847      -0.214       0.261
quit_time     -1.1500      0.316     -3.635      0.000      -1.770      -0.530
pkyr           1.8525      0.336      5.517      0.000       1.194       2.511
==============================================================================
"""

In [15]:
odds = pd.DataFrame({"OR": smf_lr.params, "Lower CI": smf_lr.conf_int()[0], "Upper CI": smf_lr.conf_int()[1]})
odds = np.exp(odds)
odds["OR"] = odds["OR"].apply(lambda x: '{:.4f}'.format(x))
odds

,OR,Lower CI,Upper CI
Intercept,0.1004,0.071415,0.141193
age,3.5969,2.657615,4.868082
bmi,0.1384,0.055168,0.347345
copd,1.6540,1.234319,2.216345
phist,1.6016,1.159735,2.211861
fhist,1.2144,1.002098,1.471686
smo_status,1.0237,0.807230,1.298136
quit_time,0.3166,0.170306,0.588695
pkyr,6.3755,3.301414,12.311823


In [16]:
import math
X, y = cohort_enc[features], cohort_enc['lung_cancer'].to_numpy().ravel()
y_prob = smf_lr.predict(cohort_enc[features])
y_hat = round(y_prob)

# accuracy_score
roc = roc_auc_score(y, y_prob)
report = classification_report(y, y_hat)
acc = accuracy_score(y, y_hat)
print(f"AUC: {roc}")
print(report)
print(f"Accuracy: {acc}")
print(f"Chi-squared statistic: {smf_lr.llr} p:{smf_lr.llr_pvalue}")

AUC: 0.6679577061371582
              precision    recall  f1-score   support

           0       0.88      1.00      0.94      5113
           1       0.00      0.00      0.00       676

    accuracy                           0.88      5789
   macro avg       0.44      0.50      0.47      5789
weighted avg       0.78      0.88      0.83      5789

Accuracy: 0.8825358438417689
Chi-squared statistic: 201.6990045011653 p:2.8020091901612823e-39
